# Climb Example

In [3]:
# from cosapp.drivers import EulerExplicit, RungeKutta, NonLinearSolver, RunSingleCase
from cosapp.recorders import DataFrameRecorder
import amad.disciplines.aerodynamics.tools.createAeroInterpolationCSV as aeroInterp  # Tool to create the function to Interpolate.
from cosapp.drivers import RungeKutta, NonLinearSolver
from amad.disciplines.performance.systems.Climb import Climb_segment
import numpy as np
import amad.tools.unit_conversion as uc

In [11]:
"""Aerodynamic inputs"""
# Directory to the Aero results in CSV in order to build the interpolation functions.
Aero_CSV = r'amad/disciplines/aerodynamics/tools/Results/aero_results.csv'

# Ranges to create CL and CD interpolation functions from AVL Aero Results (CSV file).
# REMARK: To maintain units, array and split format to the values must correlate to the CSV file results.
alpha_list = list(np.arange(-6.0, 10.0, 2))  # unit='deg', desc='Range of AOA to create functions')
mach_list = list(np.arange(0.0, 0.84, 0.04))  # unit='', desc='Range of Mach to create functions')
altitude_list = list(np.arange(0.0, 13000, 500))  # unit='m', desc='Range of altitude to create functions')

# Creation of functions for the aerodynamic coefficients interpolation.
CLAeroIt = aeroInterp.CL_Interpolation_function(alpha_list, mach_list, altitude_list, Aero_CSV)
CDAeroIt = aeroInterp.CD_Interpolation_function(alpha_list, mach_list, altitude_list, Aero_CSV)
DAeroIt = aeroInterp.Drag_Interpolation_function(alpha_list, mach_list, altitude_list, Aero_CSV)

s1 = Climb_segment(name='s1')
s1.CLAeroIt = CLAeroIt
s1.CDAeroIt = CDAeroIt
s1.DAeroIt = DAeroIt

driver = s1.add_driver(RungeKutta())
solver = driver.add_child(NonLinearSolver('solver'))
driver.time_interval = (0, 10000)
driver.dt = 2
Data_to_record = [
    'Altitude', 'CD', 'CL',
    'Distance', 'Drag', 'Lift',
    'Mach', 'RC', 'TAS', 'CAS',
    'THR', 'alpha', 'gamma', 'theta',
    'mass', 'out_p.fuel_mass', 'time']

# Add a recorder to capture time evolution in a dataframe
driver.add_recorder(
    DataFrameRecorder(
        includes = Data_to_record,
    ), period=10
)

# Initial conditions
p_0 = np.array([0.0, 0.0, 437.0])  # Position [m]
CS_0 = np.array([0.0])  # Fuel consumption rate [kg/s]
m_0 = np.array([68000])  # mass [kg]

# Define a simulation scenario
driver.set_scenario(
    init = {'CS': CS_0, 'in_p.position': p_0, 'g': 9.81, 'm0': m_0},  #
    values = {
        'CAS': uc.kt2ms(250),  # [kt]
        'Iso_Mach': 0.75,
        'cruise_altitude': 10668,  # m
        'S': 124,  # [m^2]
        'n_eng': 2,
    },
)

s1.run_drivers()
data = driver.recorder.export_data()


ArithmeticError: The solver failed: Singular 1x1 Jacobian matrix